# Challenge ONE 2: Telecom X

## 📌 Extracción

In [183]:
## Imports
import pandas as pd
import json 
import requests
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [184]:
url = 'https://raw.githubusercontent.com/alura-cursos/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'
response = requests.get(url)
df = pd.json_normalize(data=json.loads(response.text))
df.head()

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


## 🔧 Transformación

### Conocer el conjunto de datos

In [185]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer.gender            7267 non-null   object 
 3   customer.SeniorCitizen     7267 non-null   int64  
 4   customer.Partner           7267 non-null   object 
 5   customer.Dependents        7267 non-null   object 
 6   customer.tenure            7267 non-null   int64  
 7   phone.PhoneService         7267 non-null   object 
 8   phone.MultipleLines        7267 non-null   object 
 9   internet.InternetService   7267 non-null   object 
 10  internet.OnlineSecurity    7267 non-null   object 
 11  internet.OnlineBackup      7267 non-null   object 
 12  internet.DeviceProtection  7267 non-null   object 
 13  internet.TechSupport       7267 non-null   objec

In [186]:
# Verificación de los valores únicos en las columnas para conocer su contenido
for column in df.columns[1:]:
    unique_values = df[column].sort_values().unique()
    if len(unique_values) < 10:
        print(column, '\n',unique_values, '\n')
    else:
        print(column, '\nMin:',min(unique_values), '\t Max:', max(unique_values), '\n')

Churn 
 ['' 'No' 'Yes'] 

customer.gender 
 ['Female' 'Male'] 

customer.SeniorCitizen 
 [0 1] 

customer.Partner 
 ['No' 'Yes'] 

customer.Dependents 
 ['No' 'Yes'] 

customer.tenure 
Min: 0 	 Max: 72 

phone.PhoneService 
 ['No' 'Yes'] 

phone.MultipleLines 
 ['No' 'No phone service' 'Yes'] 

internet.InternetService 
 ['DSL' 'Fiber optic' 'No'] 

internet.OnlineSecurity 
 ['No' 'No internet service' 'Yes'] 

internet.OnlineBackup 
 ['No' 'No internet service' 'Yes'] 

internet.DeviceProtection 
 ['No' 'No internet service' 'Yes'] 

internet.TechSupport 
 ['No' 'No internet service' 'Yes'] 

internet.StreamingTV 
 ['No' 'No internet service' 'Yes'] 

internet.StreamingMovies 
 ['No' 'No internet service' 'Yes'] 

account.Contract 
 ['Month-to-month' 'One year' 'Two year'] 

account.PaperlessBilling 
 ['No' 'Yes'] 

account.PaymentMethod 
 ['Bank transfer (automatic)' 'Credit card (automatic)' 'Electronic check'
 'Mailed check'] 

account.Charges.Monthly 
Min: 18.25 	 Max: 118.75 

ac

**Diccionario de datos del conjunto de datos**

- `customerID`: número de identificación único de cada cliente
- `Churn`: si el cliente dejó o no la empresa
- `gender`: género (masculino y femenino)
- `SeniorCitizen`: información sobre si un cliente tiene o no una edad igual o mayor a 65 años
- `Partner`: si el cliente tiene o no una pareja
- `Dependents`: si el cliente tiene o no dependientes
- `tenure`: meses de contrato del cliente
- `PhoneService`: suscripción al servicio telefónico
- `MultipleLines`: suscripción a más de una línea telefónica
- `InternetService`: suscripción a un proveedor de internet
- `OnlineSecurity`: suscripción adicional de seguridad en línea
- `OnlineBackup`: suscripción adicional de respaldo en línea
- `DeviceProtection`: suscripción adicional de protección del dispositivo
- `TechSupport`: suscripción adicional de soporte técnico, menor tiempo de espera
- `StreamingTV`: suscripción de televisión por cable
- `StreamingMovies`: suscripción de streaming de películas
- `Contract`: tipo de contrato
- `PaperlessBilling`: si el cliente prefiere recibir la factura en línea
- `PaymentMethod`: forma de pago
- `Charges.Monthly`: total de todos los servicios del cliente por mes
- `Charges.Total`: total gastado por el cliente

### Comprobación de incoherencias en los datos

In [187]:
# Busqueda de valores nulos
for column in df.columns:
    null_values = df[column].isnull().sum()
    if null_values > 0:
        print(column, '-', null_values, '\n')
    else:
        print(column, '-', 'No null values', '\n')

customerID - No null values 

Churn - No null values 

customer.gender - No null values 

customer.SeniorCitizen - No null values 

customer.Partner - No null values 

customer.Dependents - No null values 

customer.tenure - No null values 

phone.PhoneService - No null values 

phone.MultipleLines - No null values 

internet.InternetService - No null values 

internet.OnlineSecurity - No null values 

internet.OnlineBackup - No null values 

internet.DeviceProtection - No null values 

internet.TechSupport - No null values 

internet.StreamingTV - No null values 

internet.StreamingMovies - No null values 

account.Contract - No null values 

account.PaperlessBilling - No null values 

account.PaymentMethod - No null values 

account.Charges.Monthly - No null values 

account.Charges.Total - No null values 



In [188]:
# Busqueda de valores incoherentes en customerID
print(f"Indices duplicados: {len(df[~df['customerID'].isin(df['customerID'])])}")
filtro = df['customerID'].str.match(r'\d{4}\-\D{4}')
print(f"Indices con formato incorrecto: {len(df[~filtro])}")
df['customerID'] = df['customerID'].str.upper().str.strip()

Indices duplicados: 0
Indices con formato incorrecto: 0


In [218]:
# Busqueda de valores incoherentes en Churn
filtro = df['Churn'].isin(['Yes', 'No'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")
print("Lista de valores con inconsistencias")
print(df.loc[~filtro, 'Churn'].value_counts())

Indices con valores incoherentes: 224
Lista de valores con inconsistencias
Churn
    224
Name: count, dtype: int64


In [190]:
# Buscando valores inconsistentes en customer.gender
filtro = df['customer.gender'].isin(['Male', 'Female'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [191]:
# Buscando valores inconsistentes en customer.SeniorCitizen
filtro = df['customer.SeniorCitizen'].isin([0, 1])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [192]:
# Buscando valores inconsistentes en customer.Partner
filtro = df['customer.Partner'].isin(['Yes', 'No'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [193]:
# Buscando valores inconsistentes en customer.Dependents
filtro = df['customer.Dependents'].isin(['Yes', 'No'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [194]:
# Buscando valores inconsistentes en customer.tenure
filtro = df['customer.tenure'] < 0
print(f"Indices con valores incoherentes: {len(df[filtro])}")

Indices con valores incoherentes: 0


In [195]:
# Buscando valores inconsistentes en phone.PhoneService
filtro = df['phone.PhoneService'].isin(['Yes', 'No'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [196]:
# Buscando valores inconsistentes en phone.MultipleLines
filtro = df['phone.MultipleLines'].isin(['Yes', 'No', 'No phone service'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [197]:
# Buscando valores inconsistentes en internet.InternetService
filtro = df['internet.InternetService'].isin(['DSL', 'Fiber optic', 'No'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [198]:
# Buscando valores inconsistentes en internet.OnlineSecurity
filtro = (~df['internet.OnlineSecurity'].isin(['Yes', 'No', 'No internet service'])) | ((df['internet.InternetService'] == 'No') & (df['internet.OnlineSecurity'] != 'No internet service'))
print(f"Indices con valores incoherentes: {len(df[filtro])}")

Indices con valores incoherentes: 0


In [199]:
# Buscando valores inconsistentes en internet.OnlineBackup
filtro = (~df['internet.OnlineBackup'].isin(['Yes', 'No', 'No internet service'])) | ((df['internet.InternetService'] == 'No') & (df['internet.OnlineBackup'] != 'No internet service'))
print(f"Indices con valores incoherentes: {len(df[filtro])}")

Indices con valores incoherentes: 0


In [200]:
# Buscando valores inconsistentes en internet.DeviceProtection
filtro = (~df['internet.DeviceProtection'].isin(['Yes', 'No', 'No internet service'])) | ((df['internet.InternetService'] == 'No') & (df['internet.DeviceProtection'] != 'No internet service'))
print(f"Indices con valores incoherentes: {len(df[filtro])}")


Indices con valores incoherentes: 0


In [201]:
# Buscando valores inconsistentes en internet.TechSupport
filtro = (~df['internet.TechSupport'].isin(['Yes', 'No', 'No internet service'])) | ((df['internet.InternetService'] == 'No') & (df['internet.TechSupport'] != 'No internet service'))
print(f"Indices con valores incoherentes: {len(df[filtro])}")

Indices con valores incoherentes: 0


In [202]:
# Buscando valores inconsistentes en internet.StreamingTV
filtro = (~df['internet.StreamingTV'].isin(['Yes', 'No', 'No internet service'])) | ((df['internet.InternetService'] == 'No') & (df['internet.StreamingTV'] != 'No internet service'))
print(f"Indices con valores incoherentes: {len(df[filtro])}")

Indices con valores incoherentes: 0


In [ ]:
# Buscando valores inconsistentes en internet.StreamingMovies
#filtro = df['internet.StreamingMovies'].isin(['Yes', 'No', 'No internet service'])
filtro = (~df['internet.StreamingMovies'].isin(['Yes', 'No', 'No internet service'])) | ((df['internet.InternetService'] == 'No') & (df['internet.StreamingMovies'] != 'No internet service'))
print(f"Indices con valores incoherentes: {len(df[filtro])}")


Indices con valores incoherentes: 0


In [204]:
# Buscando valores inconsistentes en account.Contract
filtro = df['account.Contract'].isin(['Month-to-month', 'One year', 'Two year'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [205]:
# Buscando valores inconsistentes en account.PaperlessBilling
filtro = df['account.PaperlessBilling'].isin(['Yes', 'No'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [206]:
# Buscando valores inconsistentes en account.PaymentMethod
filtro = df['account.PaymentMethod'].isin(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'])
print(f"Indices con valores incoherentes: {len(df[~filtro])}")

Indices con valores incoherentes: 0


In [207]:
# Buscando valores inconsistentes en account.Charges.Monthly
print(f"Min. Values: {df['account.Charges.Monthly'].min()}, Max. Values: {df['account.Charges.Monthly'].max()}")

Min. Values: 18.25, Max. Values: 118.75


In [217]:
# Buscando valores inconsistentes en account.Charges.Total
filtro = (~df['account.Charges.Total'].str.match(r"\d+(?:\.\d+)?")) | (df['account.Charges.Total'].isna())
print(f"Indices con valores incoherentes: {len(df[filtro])}")
print("Lista de valores con inconsistencias")
print(df.loc[filtro, 'account.Charges.Total'].value_counts())


Indices con valores incoherentes: 11
Lista de valores con inconsistencias
account.Charges.Total
    11
Name: count, dtype: int64


In [210]:
# Completar número sin decimales
#df['account.Charges.Total'] = df['account.Charges.Total'].replace('\.$', '.0', regex=True)
# Llenar valores sin total
#filtro = df['account.Charges.Total'].astype(str).str.match(r'^\d+\.?\d?$')
#df.loc[~filtro, 'account.Charges.Total'] = df['customer.tenure'] * df['account.Charges.Monthly']

# Convertir a tipo numérico
#df['account.Charges.Total'] = df['account.Charges.Total'].astype(np.float64)

## 📊 Carga y análisis

## 📄Informe final